# **Machine Learning Applications Final Project Dash**

Date: May 9, 2023

Authors: Óscar Montoya Amores (100451858), Javier Serrano Sánchez-Ocaña (100451673), Miguel Ángel Ponce Martínez (100451309)

#### **LIBRARIES IMPORT**

In [1]:
import joblib

from jupyter_dash import JupyterDash
from dash import dash, dcc, html, Input, Output

import plotly.express as px
import pandas as pd
import numpy as np
# import gensim
import pyLDAvis.gensim as gensimvis
import pyLDAvis

PLOTS A IMPLEMENTAR, BORRAR CUANDO ESTE ACABADO:

-LDA CON LOS TÓPICOS DEPENDIENDO DE CADA GENERO, ES DECIR PODER ELEGIR QUE GENERO PLOTEAR Y QUE SALGAN LAS PALABRAS Q CONFORMAN EL TOPICO

In [2]:
df = pd.read_csv('all_lyrics.csv')
print(df.columns)

ldag = joblib.load("lda_model.joblib")
hdp = joblib.load("hdp_model.joblib")

from gensim.corpora import Dictionary

dictionary_file = "dictionary.gensim"
D = Dictionary.load(dictionary_file)
from gensim import corpora
# load the corpus from file
corpus = corpora.MmCorpus('corpus.mm')

corpus_words = [[D[word_id] for word_id, count in doc] for doc in corpus]

print(type(ldag))
print(type(corpus))
print(corpus_words[1])
print(type(D))
print("wa" in corpus_words[0])

# Visualize topics using pyLDAvis
vocab = {D[i]: i for i in range(len(D))}
vocab2 = {i: D[i] for i in range(len(D))}
term_frequency = {D[i]: freq for i, freq in D.cfs.items()}
term_frequency = pd.Series(term_frequency)
vis = gensimvis.prepare(ldag, corpus, D, mds='tsne', vocab=vocab2, sort_topics=False, term_frequency=term_frequency)

Index(['song', 'artist', 'type', 'lyrics'], dtype='object')


c:\Users\34649\anaconda3\envs\ML\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


<class 'gensim.models.ldamodel.LdaModel'>
<class 'gensim.corpora.mmcorpus.MmCorpus'>
['call', 'came', 'come', 'cry', 'feeling', 'get', 'go', 'gold', 'head', 'heaven', 'know', 'leaving', 'long', 'look', 'make', 'new', 'one', 'ooh', 'sometimes', 'soon', 'soul', 'spirit', 'stand', 'sure', 'thought', 'time', 'two', 'u', 'walk', 'want', 'white', 'wonder', 'yes', '50', '60', '70', 'ah', 'air', 'alabama', 'albatross', 'along_watchtower', 'already', 'among', 'ancient', 'anymore', 'apple', 'art_thou', 'ashford', 'away', 'ay', 'aye_aye', 'back', 'bad', 'band', 'bark', 'bat', 'battle', 'behind', 'berry', 'bird', 'bird_paradise', 'biv', 'black', 'blue', 'bob_dylan', 'body', 'bone', 'book', 'boy', 'bring', 'brings', 'brown', 'bunch', 'cannot_handle', 'caribbean', 'caught', 'caught_glimpse', 'changed', 'child', 'china', 'choir', 'chorus', 'church', 'circle', 'classify', 'code', 'colliding', 'color', 'comic_book', 'confuse', 'congregation', 'consequence', 'cover', 'crash', 'crazy', 'crime', 'dance', 

c:\Users\34649\anaconda3\envs\ML\lib\site-packages\pyLDAvis\_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  head(R).drop('saliency', 1)
c:\Users\34649\anaconda3\envs\ML\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\34649\anaconda3\envs\ML\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
c:\Users\34649\anaconda3\envs\ML\lib\site-packages\sklearn\manifold\_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be

In [3]:
# Initialize the Dash app
app = JupyterDash(__name__)

server = app.server

# Define the layout of the Dash app
app.layout = html.Div([
    html.H1("Song Genre Topic Modeling"),
    dcc.Dropdown(
        id='dropdown',
        options=[{'label': 'Topic {}'.format(i), 'value': i} for i in range(ldag.num_topics)],
        value=0
    ),
    dcc.Graph(id='topic-type-plot'),
    html.Iframe(
        id='pyldavis',
        srcDoc=pyLDAvis.prepared_data_to_html(vis),
        style={'width': '100%', 'height': '500px'}
    )
])

# Define the update functions for the Dash app
@app.callback(
    Output('pyldavis', 'srcDoc'),
    Input('dropdown', 'value')
)
def update_pyldavis(selected_topic):
    return pyLDAvis.prepared_data_to_html(vis)

@app.callback(
    Output('topic-type-plot', 'figure'),
    Input('dropdown', 'value')
)
def update_topic_type_plot(selected_topic):
    # Get the top 20 words for the selected topic
    top_words = ldag.show_topic(selected_topic, topn=2)
    print(top_words)
    top_words = [word[0] for word in top_words]
    if "wa" in top_words:
        top_words.remove("wa")
    print(top_words)
    # Find the indices of the songs in the corpus that contain any of the top 20 words for the selected topic
    indices = []
    print(len(corpus_words))
    """
    i = 0
    while i<len(corpus_words):
        for word in top_words:
            if word!="wa":
                if word in corpus_words[i]:
                    #print(word)
                    indices.append(i)
                    break
        i+=1
    """

    for i in range(len(corpus_words)):
        j=0
        for word in top_words:
            if word in corpus_words[i]:
                j+=1
        if j==len(top_words):
            indices.append(i)
            
    print(indices)
    # Filter the dataframe to include only songs with indices in the list of indices
    topic_df = df[df.index.isin(indices)]
    print(topic_df)
    # Check if any songs match the selected topic
    if topic_df.empty:
        return {'data': [], 'layout': {}}
    # Get the genre of each song in the selected topic
    genres = [genre for genre in topic_df['type']]
    # Count the number of songs in each genre for the selected topic
    genre_counts = pd.Series(genres).value_counts().sort_index()
    # Create the plot
    fig = {
        'data': [{'x': genre_counts.index, 'y': genre_counts.values, 'type': 'bar', 'marker': {'color': ['red', 'green', 'blue', 'orange']}}],
        'layout': {'title': f'Topic {selected_topic} {top_words} Song Genres by LDA' ,'xaxis': {'title': 'Genre'}, 'yaxis': {'title': 'Count'}}
    }
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



c:\Users\34649\anaconda3\envs\ML\lib\site-packages\jupyter_dash\jupyter_app.py:310: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  thread.setDaemon(True)


[('wa', 0.022217337), ('one', 0.010883816)]
['one']
3731
[0, 1, 7, 16, 17, 21, 22, 23, 28, 29, 31, 32, 38, 39, 42, 45, 46, 53, 54, 55, 57, 62, 64, 68, 69, 70, 72, 73, 74, 78, 79, 85, 86, 87, 88, 92, 95, 96, 99, 101, 103, 107, 109, 113, 115, 118, 121, 122, 124, 126, 127, 129, 131, 135, 143, 148, 149, 150, 155, 156, 157, 160, 161, 163, 166, 167, 168, 169, 179, 180, 182, 186, 188, 195, 198, 200, 204, 210, 214, 216, 218, 221, 222, 223, 227, 229, 231, 233, 235, 237, 240, 244, 249, 253, 255, 257, 258, 264, 265, 272, 273, 275, 276, 279, 282, 283, 287, 289, 290, 293, 295, 297, 298, 304, 306, 310, 311, 319, 324, 326, 333, 334, 335, 343, 344, 345, 346, 347, 350, 355, 356, 362, 363, 365, 367, 368, 379, 381, 389, 390, 391, 396, 402, 404, 405, 414, 415, 416, 417, 418, 420, 421, 423, 431, 433, 437, 440, 442, 443, 444, 447, 450, 453, 457, 462, 464, 469, 470, 471, 473, 477, 479, 480, 482, 483, 484, 485, 487, 494, 497, 498, 504, 507, 508, 512, 513, 516, 517, 529, 530, 531, 533, 535, 538, 541, 542, 545,

[('wa', 0.022217337), ('one', 0.010883816)]
['one']
3731
[0, 1, 7, 16, 17, 21, 22, 23, 28, 29, 31, 32, 38, 39, 42, 45, 46, 53, 54, 55, 57, 62, 64, 68, 69, 70, 72, 73, 74, 78, 79, 85, 86, 87, 88, 92, 95, 96, 99, 101, 103, 107, 109, 113, 115, 118, 121, 122, 124, 126, 127, 129, 131, 135, 143, 148, 149, 150, 155, 156, 157, 160, 161, 163, 166, 167, 168, 169, 179, 180, 182, 186, 188, 195, 198, 200, 204, 210, 214, 216, 218, 221, 222, 223, 227, 229, 231, 233, 235, 237, 240, 244, 249, 253, 255, 257, 258, 264, 265, 272, 273, 275, 276, 279, 282, 283, 287, 289, 290, 293, 295, 297, 298, 304, 306, 310, 311, 319, 324, 326, 333, 334, 335, 343, 344, 345, 346, 347, 350, 355, 356, 362, 363, 365, 367, 368, 379, 381, 389, 390, 391, 396, 402, 404, 405, 414, 415, 416, 417, 418, 420, 421, 423, 431, 433, 437, 440, 442, 443, 444, 447, 450, 453, 457, 462, 464, 469, 470, 471, 473, 477, 479, 480, 482, 483, 484, 485, 487, 494, 497, 498, 504, 507, 508, 512, 513, 516, 517, 529, 530, 531, 533, 535, 538, 541, 542, 545,